In [286]:
import os
import gilda
import pickle
import pandas as pd
from indra.sources import tas
from collections import defaultdict
import indra.tools.assemble_corpus as ac
from indra.ontology.bio import bio_ontology
from indra.sources import indra_db_rest as idr
from indra.assemblers.html import HtmlAssembler
from indra.ontology.standardize import standardize_db_refs

In [287]:
def html_assembler(indra_stmts, fname):
    """Assemble INDRA statements into a HTML report"""
    html_assembler = HtmlAssembler(indra_stmts,
                                   db_rest_url='https://db.indra.bio')
    assembled_html_report = html_assembler.make_model(no_redundancy=True)
    html_assembler.save_model(fname)
    return assembled_html_report

In [288]:
HERE = os.path.join(os.getcwd())
top_hits_df = pd.read_csv(os.path.join(HERE, 'input', 'okl_top_hits.csv'))

if not os.path.isfile(os.path.join(HERE, 'input', 'tas.pkl')):
    tp = tas.process_from_web()
    with open(os.path.join(HERE, 'input', 'tas.pkl'), 'wb') as fh:
        pickle.dump(tp, fh)
else:
    with open(os.path.join(HERE, 'input', 'tas.pkl'), 'rb') as fh:
        tp = pickle.load(fh)

In [289]:
compound_names = [compounds.split()[0] for compounds in top_hits_df['Compound Name']]
compound_targets = defaultdict(set)
tas_stmts = []

chembl_ids = {
    'CHEMBL214253': 'PD-407824',
    'CHEMBL254760': 'MGCD265',
    'CHEMBL363648': 'TAK-715',
    'CHEMBL373751': 'KIN001-135',
    'CHEMBL603469': 'Lestaurtinib',
    'CHEMBL377300': 'Brivanib',
    'CHEMBL565612': 'Sotrastaurin',
    'CHEMBL571948': 'Y39983',
    'CHEMBL2028663': 'Dabrafenib',
    'CHEMBL1078178': 'Momelotinib'   
}

# Get target statements
for stmt in tp.statements:
    subj = stmt.subj.name
    obj = stmt.obj.name
    if subj in compound_names:
        compound_targets[(subj)].add(obj)
        tas_stmts.append(stmt)
    elif stmt.subj.db_refs.get('CHEMBL') in chembl_ids:
        compound_targets[(chembl_ids[stmt.subj.db_refs.get('CHEMBL')])].add(obj)


# Create a new column for tas targets
top_hits_df['tas_targets'] = None

# Update tas_targets with hits
for r,c in top_hits_df.iterrows():
    cname = c[3].split()[0]
    if cname in compound_targets:
        top_hits_df['tas_targets'][r] = ", ".join(compound_targets[cname])

# Write the dataframe to a new csv
top_hits_df.to_csv(os.path.join(HERE, 'output', 'tas_hits.csv'))

indra_stmts = ac.run_preassembly(tas_stmts,
                                 run_refinement=False)

# Assemble the statements into HTML formatted report and save into a file
indra_op_html_report = \
    html_assembler(
        indra_stmts,
        fname=os.path.join(HERE, 'output', 'tas_statements.html'))

<ipython-input-289-8fb84b2b028b>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_hits_df['tas_targets'][r] = ", ".join(compound_targets[cname])
INFO: [2021-08-24 16:51:16] indra.tools.assemble_corpus - Combining duplicates on 358 statements...
INFO: [2021-08-24 16:51:16] indra.tools.assemble_corpus - 358 unique statements


In [290]:
## INDRA DB SEARCH

indra_stmts = []
indra_hits = defaultdict(set)


for r, c in top_hits_df.iterrows():
    # Grounding subject using Gilda
    subj = c[3].split()[0]

    gilda_subj = gilda.ground(subj)
    gilda_subj = gilda_subj[0].term.entry_name if gilda_subj else 'NA'
    if gilda_subj == 'KW 2449':
        gilda_subj = 'KW2449'
    # Downloading statements using INDRA REST API
    if gilda_subj != 'NA':
        idrp = idr.get_statements(subject=gilda_subj)
        indra_stmts = indra_stmts + idrp.statements
        for stmts in idrp.statements:
            if type(stmts).__name__ == 'Inhibition':
                subj = stmts.subj.name
                obj = stmts.obj.name
                indra_hits[(c[3].split()[0])].add(obj)
    elif gilda_subj == 'NA':
        indra_hits[(c[3].split()[0])].add('NA')
        
# Create a new column for indra_db targets
top_hits_df['indra_db_targets'] = None

# Update indra_db_targets with hits
for r,c in top_hits_df.iterrows():
    cname = c[3].split()[0]
    if cname in indra_hits:
        top_hits_df['indra_db_targets'][r] = ", ".join(indra_hits[cname])

# Write the dataframe to a new csv
top_hits_df.to_csv(os.path.join(HERE, 'output', 'tas_indra_db_hits.csv'))


indra_stmts = ac.run_preassembly(indra_stmts,
                                 run_refinement=False)

# Assemble the statements into HTML formatted report and save into a file
indra_op_html_report = \
    html_assembler(
        indra_stmts,
        fname=os.path.join(HERE, 'output', 'indra_db_statements.html'))

INFO: [2021-08-24 16:51:27] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=PD 407824 with role=SUBJECT.
INFO: [2021-08-24 16:51:27] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2021-08-24 16:51:27] indra_db_rest.request_logs -   LIMIT: None
INFO: [2021-08-24 16:51:27] indra_db_rest.request_logs -   OFFSET: 0
INFO: [2021-08-24 16:51:28] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=LY-2157299 with role=SUBJECT.
INFO: [2021-08-24 16:51:28] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2021-08-24 16:51:28] indra_db_rest.request_logs -   LIMIT: None
INFO: [2021-08-24 16:51:28] indra_db_rest.request_logs -   OFFSET: 0
INFO: [2021-08-24 16:51:29] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=A-674563 with role=SUBJECT.
INFO: [2021-08-24 16:51:29] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2021-08-24 16:

In [291]:
# Standardize namespaces
all_chembl_ids = {
    'CEP-1347' : 'CHEMBL290352',
    'PD-407824':'CHEMBL214253',
    'Galunisertib': 'CHEMBL2364611',
    'A-674563': 'CHEMBL379218',
    'MGCD265': 'CHEMBL254760',
    'TAK-715': 'CHEMBL363648',
    'KIN001-135': 'CHEMBL373751',
    'Lestaurtinib': 'CHEMBL603469',
    'CYC116': 'CHEMBL482967',
    'KW2449': 'CHEMBL1908397',
    'Brivanib': 'CHEMBL377300',
    'Sotrastaurin': 'CHEMBL565612',
    'Y39983': 'CHEMBL571948',
    'Dabrafenib': 'CHEMBL2028663',
    'Momelotinib': 'CHEMBL1078178',
    'Y-27632': 'CHEMBL559147'
}



# Create a new column for standard namespaces
top_hits_df['standard_db_refs'] = None 
for r,c in top_hits_df.iterrows():
    cname = c[3].split()[0]
    if cname in all_chembl_ids:
        d = standardize_db_refs({'CHEMBL':all_chembl_ids[cname]})
        d = [(k+":"+v) for k,v in d.items()]
        top_hits_df['standard_db_refs'][r] = ", ".join(d)

<ipython-input-291-4f5a0ec9db5e>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_hits_df['standard_db_refs'][r] = ", ".join(d)


In [295]:
# Add toxic drugs
toxic_drugs = ['Y', 'Y', 'N', 'N','N','N','N','N', 'Y', 'N','N','N','N', 'Y', 'N', 'Y', 'N', 'N', 'N']
top_hits_df['toxic'] = candidate_drugs

# Add fav drugs
fav_hits = ['N','N','Y','N','Y', 'N','N','N','N','N','Y', 'Y', 'N','Y','N','N','N','Y', 'Y']
top_hits_df['favorite'] = fav_hits

# Rearrange columns
new_df = top_hits_df[['Vendor Name', 'Vendor Identifier', 'Molar Concentration',
                     'Compound Name', 'toxic', 'favorite', 'standard_db_refs', 'tas_targets', 'indra_db_targets']]
new_df

# Save df
new_df.to_csv(os.path.join(HERE, 'output', 'tas_indra_db_hits.csv'))

In [325]:
indra_stmts = []
indra_hits = defaultdict(set)
kw_hits = defaultdict(set)

for r, c in top_hits_df.iterrows():
    # Grounding subject using Gilda
    subj = c[3].split()[0]

    gilda_subj = gilda.ground(subj)
    gilda_subj = gilda_subj[0].term.entry_name if gilda_subj else 'NA'
    if gilda_subj == 'KW 2449':
        gilda_subj = 'KW2449'
    # Downloading statements using INDRA REST API
    if gilda_subj != 'NA':
        idrp = idr.get_statements(subject=gilda_subj)
        indra_stmts = indra_stmts + idrp.statements
        for stmts in idrp.statements:
            if type(stmts).__name__ == 'Inhibition':
                subj = stmts.subj.name
                obj = stmts.obj.name
                if subj == 'KW2449':
                    kw_hits[(c[3].split()[0])].add(obj)
                indra_hits[(c[3].split()[0])].add(obj)
    elif gilda_subj == 'NA':
        indra_hits[(c[3].split()[0])].add('NA')

INFO: [2021-08-31 10:17:48] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=PD 407824 with role=SUBJECT.
INFO: [2021-08-31 10:17:48] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2021-08-31 10:17:48] indra_db_rest.request_logs -   LIMIT: None
INFO: [2021-08-31 10:17:48] indra_db_rest.request_logs -   OFFSET: 0
INFO: [2021-08-31 10:17:49] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=LY-2157299 with role=SUBJECT.
INFO: [2021-08-31 10:17:49] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2021-08-31 10:17:49] indra_db_rest.request_logs -   LIMIT: None
INFO: [2021-08-31 10:17:49] indra_db_rest.request_logs -   OFFSET: 0
INFO: [2021-08-31 10:17:50] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=A-674563 with role=SUBJECT.
INFO: [2021-08-31 10:17:50] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2021-08-31 10:

In [339]:
fh = open(os.path.join(HERE, 'output', 'kw_hits.txt'), 'w')
[fh.write(v1+'\n') for i,v in kw_hits.items() for v1 in v]
fh.close()


In [324]:
kw_hits.

<_io.TextIOWrapper name='/Users/sbunga/gitHub/panacea_indra/panacea_exam/drug_target_search/output/kw_hits.txt' mode='w' encoding='UTF-8'>